<a href="https://colab.research.google.com/github/slifer4195/bertTransfer/blob/main/in1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ktrain


In [33]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers


labels = []           #labels
title = []          #title

#reading training sample but must downsample
cnt = 0
with open("train.csv", "r", encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    print("reading csv file......")
    print("Please wait for a while as the csv file is very long.")
    for line in csv_reader:
        labels.append(int(line[0]))
        title.append(line[1] + ' '+ line[2]+ ' '+line[3])
        cnt += 1
        if cnt == 10000: break;

#this is to read all the classfications for labels
classes =[]
with open("classes.txt", "r", encoding='utf-8') as txt:
    print("reading classes.txt")
    for line in txt:
        classes.append(line)

#this function remove urls
def removing_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

#remove punct
def removing_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

stop = set(stopwords.words('english'))
def removing_stopewords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

title = list(map(removing_URL, title))
title = list(map(removing_punct, title))
title = list(map(removing_stopewords, title))


val_x = []           #labels
val_Y = []          #title

#reading training sample but must downsample
cnt = 0
with open("test.csv", "r", encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    print("reading csv test file......")
    print("Please wait for a while as the csv file is very long.")
    for line in csv_reader:
        val_Y.append(int(line[0]))
        val_x.append(line[1] + ' '+ line[2]+ ' '+line[3])
        cnt += 1
        if cnt == 2000: break;



#Count unique words kinda like diction
def counter_word(text_col):
    count = Counter()
    for text in text_col:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(title)

train_size = int(len(title) * 0.8)

train_df = title[:train_size]  #70percent valid
val_df = title[train_size:]    #30percent valid
trainLabel_df = labels[:train_size]  #70percent valid
valLabel_df = labels[train_size:]   #30percent valid

#you can use val_df, valLabel-df
#val_x and val_y are the data from test.csv
train_y = []
val_y = []

for i in range(len(trainLabel_df)):
  train_y.append(classes[trainLabel_df[i]-1])

for i in range(len(val_Y)):
  val_y.append(classes[val_Y[i] - 1])

classy = [0,1,2,3,4,5,6,7,8,9]
model_name = 'distilbert-base-uncased'
trans = text.Transformer(model_name,maxlen=512, class_names = classy)
train_data = trans.preprocess_train(train_df, train_y)
test_data = trans.preprocess_test(val_x, val_y)


#using pretrained network distilbert

model = trans.get_classifier()

learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data,batch_size = 16)

learner.fit_onecycle(1e-4, 3)

























[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
reading classes.txt
reading csv test file......
Please wait for a while as the csv file is very long.
preprocessing train...
language: en
train sequence lengths:
	mean : 43
	95percentile : 129
	99percentile : 281


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 81
	95percentile : 244
	99percentile : 506




begin training using onecycle policy with max lr of 0.0001...
Epoch 1/3
500/500 [==============================] - 676s 1s/step - loss: 1.7471 - accuracy: 0.4303 - val_loss: 1.0005 - val_accuracy: 0.6769
Epoch 2/3
500/500 [==============================] - 669s 1s/step - loss: 0.9738 - accuracy: 0.6892 - val_loss: 0.8117 - val_accuracy: 0.7418
Epoch 3/3
500/500 [==============================] - 669s 1s/step - loss: 0.6054 - accuracy: 0.8123 - val_loss: 0.6360 - val_accuracy: 0.8007
